In [1]:
import pandas as pd
import pandas.io.sql as psql
import os
import sys
from datetime import date
current_date = date.today()
#%load_ext autotime
import re
import csv
import pandas as pd
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
#from matplotlib_venn import venn2, venn3
import warnings
import time
import datetime as dt
import itertools
from more_itertools import unique_everseen
warnings.filterwarnings("ignore")
#import pandasql as ps
import seaborn as sns
import json
from ast import literal_eval
import ast

In [2]:
# STEPS to get data

# get arr of notes
# get arr of keywords
# split note in arr
# for each word in keyword list, get count of it in arr (word_list.count(curr_keyword))
# also store which keywords are matched in arr
# have df (with cols patient id, encounter id, note TXT, keyword count, list of flagged keywords)
# to get # of general encouters --> do len(df[df["keyword_count"] >= 1])
# to get # of such encounters per patient
    # get list of unique patient ids
    # for each one, do len(df[df["patient_id"] == curr_patient_id]["encounter_id"])
    
# REPRESENTING data

# exported dataframe to csv with cols patient_id, num encounters keywords >= 1
# exported dataframe to csv that has general encounters where keywords >= 1, cols: patient id, encounter id, note txt, keyword count, matched keywords
# Summary Statistics: number of encounters with keywords >= 1, number of average encounters per patient with such encounters

In [136]:
#df1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Mayuresh\APOE33\apoe33_notes_1.csv")
#df2 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Mayuresh\APOE33\apoe33_notes_2.csv")
#df3 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Mayuresh\APOE33\apoe33_notes_3.csv")
df4 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Mayuresh\APOE33\apoe33_notes_4.csv")
# df5 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Mayuresh\APOE33\apoe33_notes_5.csv")

In [137]:
#frames = [df1, df2]
notes = df3

In [138]:
notes["NoteTXT"] = notes["NoteTXT"].str.replace('\s+', ' ', regex=True)
notes['NoteTXT'] = '-------------------------------------------------------------------------------------------------------------' + notes['NoteTXT'].astype(str)+'-------------------------------------------------------------------------------------------------------------'

In [139]:
regex = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\Sentence-Analysis\EDA\Getting_Data\keywords.csv")

In [140]:
regex

,REGEX
0,dementia
1,cognition
2,memory
3,CDR
4,MMSE
5,MOCA
6,Alzheimer
7,cognitive\s*impairment
8,MCI
9,cerebellar


In [141]:
notes

,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT
0,101641676,10070410567,Z11443906,3.037928e+09,2015-03-03 00:00:00.0000000,Hospital Encounter,296306708,Discharge Summary,3,2.835385e+08,1.0,----------------------------------------------...
1,101641676,10070410567,Z11443906,3.048279e+09,2000-04-28 00:00:00.0000000,System Generated,330469263,Op Note,3,3.163476e+08,1.0,----------------------------------------------...
2,101641676,10070410567,Z11443906,3.048281e+09,2000-04-25 00:00:00.0000000,System Generated,330471221,Op Note,3,3.163495e+08,1.0,----------------------------------------------...
3,101641676,10070410567,Z11443906,3.050240e+09,2012-07-30 00:00:00.0000000,System Generated,333554108,Discharge Summary,3,3.193679e+08,1.0,----------------------------------------------...
4,101641676,10070410567,Z11443906,3.050240e+09,2012-07-30 00:00:00.0000000,System Generated,333554111,Discharge Summary,3,3.193679e+08,1.0,----------------------------------------------...
...,...,...,...,...,...,...,...,...,...,...,...,...
1055832,104241704,11454931,Z8126942,3.336745e+09,2020-11-11 00:00:00.0000000,Letter (Out),4843019479,Letter,3,4.833327e+09,1.0,----------------------------------------------...
1055833,104241704,11454931,Z8126942,3.346831e+09,2021-01-08 00:00:00.0000000,Telephone,5123475001,Telephone Encounter,3,5.115927e+09,1.0,----------------------------------------------...
1055834,104241704,11454931,Z8126942,3.347040e+09,2021-01-11 00:00:00.0000000,Telephone,5129063875,Telephone Encounter,3,5.121573e+09,1.0,----------------------------------------------...
1055835,104241704,11454931,Z8126942,3.359127e+09,2021-03-19 00:00:00.0000000,Office Visit,5493916523,Progress Notes,3,5.489162e+09,1.0,----------------------------------------------...


In [142]:
locations = []
for r in tqdm(notes['NoteTXT']):
    try:
        locations.append([match.span() \
                  for match in re.finditer(re.compile("|".join(regex["REGEX"]), 
                                                  re.IGNORECASE),r)])
    except:
        locations.append([])

notes["regex_location"] = locations


100%|██████████████████████████████████████████████████████████████████████| 1055837/1055837 [14:22<00:00, 1223.82it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'Regex_match_csv/3/apoe33_notes_3_w_locations.csv'

In [143]:
notes.to_csv("Regex_match_csv/3/apoe33_notes_3_w_locations.csv")

In [144]:
notes.head(100)

,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location
0,101641676,10070410567,Z11443906,3.037928e+09,2015-03-03 00:00:00.0000000,Hospital Encounter,296306708,Discharge Summary,3,2.835385e+08,1.0,----------------------------------------------...,[]
1,101641676,10070410567,Z11443906,3.048279e+09,2000-04-28 00:00:00.0000000,System Generated,330469263,Op Note,3,3.163476e+08,1.0,----------------------------------------------...,[]
2,101641676,10070410567,Z11443906,3.048281e+09,2000-04-25 00:00:00.0000000,System Generated,330471221,Op Note,3,3.163495e+08,1.0,----------------------------------------------...,[]
3,101641676,10070410567,Z11443906,3.050240e+09,2012-07-30 00:00:00.0000000,System Generated,333554108,Discharge Summary,3,3.193679e+08,1.0,----------------------------------------------...,[]
4,101641676,10070410567,Z11443906,3.050240e+09,2012-07-30 00:00:00.0000000,System Generated,333554111,Discharge Summary,3,3.193679e+08,1.0,----------------------------------------------...,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,101641676,10070410567,Z11443906,3.285137e+09,2019-12-13 00:00:00.0000000,Telephone,3379046458,Telephone Encounter,3,3.357403e+09,1.0,----------------------------------------------...,[]
96,101641676,10070410567,Z11443906,3.286428e+09,2020-01-27 00:00:00.0000000,Hospital Encounter,3586433206,Progress Notes,3,3.568232e+09,1.0,----------------------------------------------...,[]
97,101641676,10070410567,Z11443906,3.313784e+09,2020-07-02 00:00:00.0000000,Documentation,4202452850,Progress Notes,3,4.187624e+09,1.0,----------------------------------------------...,[]
98,101641676,10070410567,Z11443906,3.315734e+09,2020-07-15 00:00:00.0000000,Hospital Encounter,4261598727,Progress Notes,3,4.247263e+09,1.0,----------------------------------------------...,[]


In [145]:
l = []
for i in tqdm(range(len(notes["regex_location"]))):
    s = []
    try:
        for j in notes["regex_location"][i]:
            s.append(notes["NoteTXT"][i][j[0]-100 : j[0]+100])
    except:
        pass
    l.append(s)
    
    
notes["regex_sent"] = l

100%|█████████████████████████████████████████████████████████████████████| 1055837/1055837 [00:49<00:00, 21394.71it/s]


In [109]:
notes.to_csv("apoe33_notes_3_w_locations_and_sequences.csv")

In [110]:
notes.head(100)

,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location,regex_sent
0,100365175,10059358563,Z8714148,3.188337e+09,2018-02-01 00:00:00.0000000,Refill,1846612577,Telephone Encounter,2,1.800346e+09,1.0,----------------------------------------------...,[],[]
1,100365175,10059358563,Z8714148,3.191433e+09,2018-02-27 00:00:00.0000000,Telephone,1878601169,Telephone Encounter,2,1.833249e+09,1.0,----------------------------------------------...,[],[]
2,100365175,10059358563,Z8714148,3.191433e+09,2018-02-27 00:00:00.0000000,Telephone,1888118954,Telephone Encounter,2,1.843048e+09,1.0,----------------------------------------------...,[],[]
3,100365175,10059358563,Z8714148,3.191433e+09,2018-02-27 00:00:00.0000000,Telephone,1888194332,Telephone Encounter,2,1.843128e+09,1.0,----------------------------------------------...,[],[]
4,100365175,10059358563,Z8714148,3.198147e+09,2019-04-12 00:00:00.0000000,Office Visit,2412978091,Progress Notes,2,2.382364e+09,1.0,----------------------------------------------...,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,100365365,5361447,Z8715609,3.121898e+09,2016-06-15 00:00:00.0000000,Anesthesia Event,1099583515,Anesthesia Preprocedure Evaluation,2,1.048895e+09,1.0,----------------------------------------------...,[],[]
96,100365365,5361447,Z8715609,3.121898e+09,2016-06-15 00:00:00.0000000,Anesthesia Event,1100164531,Anesthesia Postprocedure Evaluation,2,1.049384e+09,1.0,----------------------------------------------...,[],[]
97,100365365,5361447,Z8715609,3.122194e+09,2016-06-23 00:00:00.0000000,Office Visit,1105724884,Progress Notes,2,1.054618e+09,1.0,----------------------------------------------...,[],[]
98,100365365,5361447,Z8715609,3.122607e+09,2016-06-22 00:00:00.0000000,Anesthesia,1100162799,NaN,2,1.049383e+09,1.0,----------------------------------------------...,[],[]


In [146]:
notes['regex_sent'] = notes['regex_sent'].astype(str)
notes['regex_sent'] = notes['regex_sent'].str.replace('[', '')
notes['regex_sent'] = notes['regex_sent'].str.replace(']', '')
notes = notes.replace(r'^\s*$', np.NaN, regex=True)

In [149]:
notes = notes[notes['regex_sent'].notna()]

In [150]:
notes['regex_sent'] = notes['regex_sent'].str.replace('"', "'")

In [151]:
notes = (notes.assign(regex_sent = notes['regex_sent'].str.split("', '"))
         .explode('regex_sent')
         .reset_index(drop=True))

In [152]:
l = []
for note in tqdm(notes["regex_sent"]):
    try:
        l.append(re.findall(re.compile("|".join(regex["REGEX"]), re.IGNORECASE), note))
    except:
        l.append([])

notes["regex_match"] = l

100%|███████████████████████████████████████████████████████████████████████| 254958/254958 [00:24<00:00, 10412.74it/s]


In [153]:
notes.to_csv("Regex_match_csv/3/apoe33_notes_3_w_everything.csv")

In [154]:
notes

,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location,regex_sent,regex_match
0,101641676,10070410567,Z11443906,3.131433e+09,2016-11-30 00:00:00.0000000,Office Visit,1336847119,Letter,3,1.275931e+09,1.0,----------------------------------------------...,"[(12357, 12366)]","'ions (617-724-2076). Best regards, Jacob Dal-...",[cognition]
1,101641676,10070410567,Z11443906,3.131433e+09,2016-11-30 00:00:00.0000000,Office Visit,1336847220,Progress Notes,3,1.275931e+09,1.0,----------------------------------------------...,"[(11906, 11915)]","'ions (617-724-2076). Best regards, Jacob Dal-...",[cognition]
2,101641676,10070410567,Z11443906,3.176428e+09,2018-10-31 00:00:00.0000000,Office Visit,2197803658,Progress Notes,3,2.170880e+09,1.0,----------------------------------------------...,"[(2647, 2653)]",'s several times a night and goes to bed late....,[memory]
3,101641676,10070410567,Z11443906,3.176428e+09,2018-10-31 00:00:00.0000000,Office Visit,2198458099,Progress Notes,3,2.170874e+09,1.0,----------------------------------------------...,"[(687, 696), (771, 777)]",' proxy: Yes Advance Care/End of Life Planning...,"[Cognition, memory]"
4,101641676,10070410567,Z11443906,3.176428e+09,2018-10-31 00:00:00.0000000,Office Visit,2198458099,Progress Notes,3,2.170874e+09,1.0,----------------------------------------------...,"[(687, 696), (771, 777)]",-2 Score: (P) 0 Cognition Negative: no evidenc...,"[Cognition, memory]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254953,104241704,11454931,Z8126942,3.239784e+09,2019-02-27 00:00:00.0000000,Hospital Encounter,2358085348,PHS IP CASE MANAGEMENT CLINICAL REVIEW,3,2.326126e+09,1.0,----------------------------------------------...,"[(27602, 27611)]","' Neurological: nonfocal exam, Fluent speech, ...",[Cognition]
254954,104241704,11454931,Z8126942,3.245085e+09,2019-07-09 00:00:00.0000000,Office Visit,2540142100,Patient Instructions,3,2.512511e+09,1.0,----------------------------------------------...,"[(4292, 4295), (4318, 4321)]","'g SGLT2 inhibitors, yeast infections around t...","[mci, mci]"
254955,104241704,11454931,Z8126942,3.245085e+09,2019-07-09 00:00:00.0000000,Office Visit,2540142100,Patient Instructions,3,2.512511e+09,1.0,----------------------------------------------...,"[(4292, 4295), (4318, 4321)]","infections around the penis are more common, e...","[mci, mci]"
254956,104241704,11454931,Z8126942,3.359127e+09,2021-03-19 00:00:00.0000000,Office Visit,5493916523,Progress Notes,3,5.489162e+09,1.0,----------------------------------------------...,"[(8866, 8875)]","'n-face-to-face), and finalizing the visit for...",[cognition]


In [155]:
notes['regex_match'] = notes['regex_match'].astype(str)
notes['regex_match'] = notes['regex_match'].str.replace("\[\]", '')
notes = notes.replace(r'^\s*$', np.NaN, regex=True)

In [156]:
notes = notes[notes['regex_match'].notna()]

In [157]:
notes['regex_sent'] = notes['regex_sent'].str.split(' ',1).str[1].str.rpartition(' ')[0]

In [158]:
import ast
notes.regex_match = notes.regex_match.apply(ast.literal_eval)

In [159]:
notes['regex_sent'] = notes['regex_sent'].apply(json.dumps)
notes['regex_match'] = notes['regex_match'].apply(json.dumps)

In [160]:
x = notes['regex_match'][97]
x = x.split(" ")
x

['["Alzheimer",', '"dementia"]']

In [161]:
print(len(x))

2


In [162]:
def regex_length(row):
    x = row['regex_match']
    x= x.split(" ")
    return len(x)

notes['match_count'] = notes.apply(regex_length, axis = 'columns')

In [ ]:
#notes['match_count'] = notes.apply(regex_length, axis='columns')

In [163]:
notes.to_csv("Regex_match_csv/3/apoe_3_FINAL.csv")

In [164]:
notes.head(100)

,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location,regex_sent,regex_match,match_count
0,101641676,10070410567,Z11443906,3.131433e+09,2016-11-30 00:00:00.0000000,Office Visit,1336847119,Letter,3,1.275931e+09,1.0,----------------------------------------------...,"[(12357, 12366)]","""(617-724-2076). Best regards, Jacob Dal-Bianc...","[""cognition""]",1
1,101641676,10070410567,Z11443906,3.131433e+09,2016-11-30 00:00:00.0000000,Office Visit,1336847220,Progress Notes,3,1.275931e+09,1.0,----------------------------------------------...,"[(11906, 11915)]","""(617-724-2076). Best regards, Jacob Dal-Bianc...","[""cognition""]",1
2,101641676,10070410567,Z11443906,3.176428e+09,2018-10-31 00:00:00.0000000,Office Visit,2197803658,Progress Notes,3,2.170880e+09,1.0,----------------------------------------------...,"[(2647, 2653)]","""several times a night and goes to bed late. H...","[""memory""]",1
3,101641676,10070410567,Z11443906,3.176428e+09,2018-10-31 00:00:00.0000000,Office Visit,2198458099,Progress Notes,3,2.170874e+09,1.0,----------------------------------------------...,"[(687, 696), (771, 777)]","""proxy: Yes Advance Care/End of Life Planning ...","[""Cognition"", ""memory""]",2
4,101641676,10070410567,Z11443906,3.176428e+09,2018-10-31 00:00:00.0000000,Office Visit,2198458099,Progress Notes,3,2.170874e+09,1.0,----------------------------------------------...,"[(687, 696), (771, 777)]","""Score: (P) 0 Cognition Negative: no evidence ...","[""Cognition"", ""memory""]",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,101642049,10070430029,Z11446577,3.176001e+09,2018-02-05 00:00:00.0000000,Office Visit,1854006611,Letter,3,1.807957e+09,1.0,----------------------------------------------...,"[(598, 604), (700, 720), (1579, 1585), (2319, ...","""cognitive impairment (MCI), multiple domains ...","[""cognitive impairment"", ""MCI"", ""CDR"", ""CDR""]",5
96,101642049,10070430029,Z11446577,3.176001e+09,2018-02-05 00:00:00.0000000,Office Visit,1854006611,Letter,3,1.807957e+09,1.0,----------------------------------------------...,"[(598, 604), (700, 720), (1579, 1585), (2319, ...","""The underlying etiology of these symptoms is ...","[""cerebrovascular""]",1
97,101642049,10070430029,Z11446577,3.176001e+09,2018-02-05 00:00:00.0000000,Office Visit,1854006611,Letter,3,1.807957e+09,1.0,----------------------------------------------...,"[(598, 604), (700, 720), (1579, 1585), (2319, ...","""MRI, but also significant atrophy suggesting ...","[""Alzheimer"", ""dementia""]",2
98,101642049,10070430029,Z11446577,3.176001e+09,2018-02-05 00:00:00.0000000,Office Visit,1854006611,Letter,3,1.807957e+09,1.0,----------------------------------------------...,"[(598, 604), (700, 720), (1579, 1585), (2319, ...","""a neurodegenerative component, more consisten...","[""Alzheimer"", ""dementia""]",2


In [31]:
# to get # of general encouters --> do len(df[df["keyword_count"] >= 1])
# to get # of such encounters per patient
    # get list of unique patient ids
    # for each one, do len(df[df["patient_id"] == curr_patient_id]["encounter_id"])

In [165]:
encounters_with_more_than_1_keyword = notes[notes['match_count'] >= 1]
encounters_with_more_than_1_keyword

,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location,regex_sent,regex_match,match_count
0,101641676,10070410567,Z11443906,3.131433e+09,2016-11-30 00:00:00.0000000,Office Visit,1336847119,Letter,3,1.275931e+09,1.0,----------------------------------------------...,"[(12357, 12366)]","""(617-724-2076). Best regards, Jacob Dal-Bianc...","[""cognition""]",1
1,101641676,10070410567,Z11443906,3.131433e+09,2016-11-30 00:00:00.0000000,Office Visit,1336847220,Progress Notes,3,1.275931e+09,1.0,----------------------------------------------...,"[(11906, 11915)]","""(617-724-2076). Best regards, Jacob Dal-Bianc...","[""cognition""]",1
2,101641676,10070410567,Z11443906,3.176428e+09,2018-10-31 00:00:00.0000000,Office Visit,2197803658,Progress Notes,3,2.170880e+09,1.0,----------------------------------------------...,"[(2647, 2653)]","""several times a night and goes to bed late. H...","[""memory""]",1
3,101641676,10070410567,Z11443906,3.176428e+09,2018-10-31 00:00:00.0000000,Office Visit,2198458099,Progress Notes,3,2.170874e+09,1.0,----------------------------------------------...,"[(687, 696), (771, 777)]","""proxy: Yes Advance Care/End of Life Planning ...","[""Cognition"", ""memory""]",2
4,101641676,10070410567,Z11443906,3.176428e+09,2018-10-31 00:00:00.0000000,Office Visit,2198458099,Progress Notes,3,2.170874e+09,1.0,----------------------------------------------...,"[(687, 696), (771, 777)]","""Score: (P) 0 Cognition Negative: no evidence ...","[""Cognition"", ""memory""]",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254953,104241704,11454931,Z8126942,3.239784e+09,2019-02-27 00:00:00.0000000,Hospital Encounter,2358085348,PHS IP CASE MANAGEMENT CLINICAL REVIEW,3,2.326126e+09,1.0,----------------------------------------------...,"[(27602, 27611)]","""Neurological: nonfocal exam, Fluent speech, S...","[""Cognition""]",1
254954,104241704,11454931,Z8126942,3.245085e+09,2019-07-09 00:00:00.0000000,Office Visit,2540142100,Patient Instructions,3,2.512511e+09,1.0,----------------------------------------------...,"[(4292, 4295), (4318, 4321)]","""SGLT2 inhibitors, yeast infections around the...","[""mci"", ""mci""]",2
254955,104241704,11454931,Z8126942,3.245085e+09,2019-07-09 00:00:00.0000000,Office Visit,2540142100,Patient Instructions,3,2.512511e+09,1.0,----------------------------------------------...,"[(4292, 4295), (4318, 4321)]","""around the penis are more common, especially ...","[""mci"", ""mci""]",2
254956,104241704,11454931,Z8126942,3.359127e+09,2021-03-19 00:00:00.0000000,Office Visit,5493916523,Progress Notes,3,5.489162e+09,1.0,----------------------------------------------...,"[(8866, 8875)]","""and finalizing the visit for this patient. 'T...","[""cognition""]",1


In [166]:
encounters_with_more_than_1_keyword.to_csv("Regex_match_csv/3/apoe_3_matches.csv")

In [167]:
unique_ids = notes["PatientID"].unique()
unique_ids = list(unique_ids)
len(unique_ids)

3741

In [168]:
summary_statistics_1 = pd.DataFrame(columns = ["patient_id", "encounters_with_keywords", "total_encounters"])
summary_statistics_1["patient_id"] = unique_ids
summary_statistics_1["encounters_with_keywords"] = [" "] * len(unique_ids)
summary_statistics_1["total_encounters"] = [" "] * len(unique_ids)
#summary_statistics_1["keywords"] = [" "] * len(unique_ids)

In [169]:
original_notes = pd.read_csv("Regex_match_csv/3/apoe33_notes_3_w_locations.csv")

In [170]:
for i in tqdm(range(len(unique_ids))):
    summary_statistics_1["encounters_with_keywords"][i] = len(notes[notes["PatientID"] == unique_ids[i]])
    summary_statistics_1["total_encounters"][i] = len(original_notes[original_notes["PatientID"] == unique_ids[i]])

100%|██████████████████████████████████████████████████████████████████████████████| 3741/3741 [07:40<00:00,  8.12it/s]


In [171]:
summary_statistics_1

,patient_id,encounters_with_keywords,total_encounters
0,Z11443906,14,112
1,Z11444674,8,120
2,Z11446577,155,59
3,Z11450172,66,149
4,Z11450620,9,87
...,...,...,...
3736,Z8109588,13,176
3737,Z8110923,56,271
3738,Z8122317,1,93
3739,Z8125592,2,9


In [172]:
summary_statistics_1.to_csv("Regex_match_csv/3/summary_statistics.csv")